In [ ]:
#!pip install --upgrade google-cloud-bigquery[bqstorage,pandas]
#!pip install google-cloud-bigquery-storage

In [ ]:
import os
from google.cloud import bigquery
from google.cloud import bigquery_storage

In [ ]:
bigquery.__version__

#### Requirements
In order to execute the lab, you need create a service account with the following roles:
- BigQuery Data Editor
- BigQuery Job User
- BigQuery Read Session User
- Storage Admin

In [ ]:
service_account = os.path.dirname(os.path.realpath('__file__')) + "/service-account.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account

In [ ]:
# Bigquery instance
bigquery_client = bigquery.Client()

#### Create dataset and table

In [ ]:
# Create dataset
dataset = bigquery_client.create_dataset(dataset="bigquery_lab")

In [ ]:
# Create table
table = dataset.table(table_id="Person")
schema = [
    bigquery.SchemaField(name="name", field_type="STRING", mode="REQUIRED"),
    bigquery.SchemaField(name="age", field_type="INTEGER", mode="REQUIRED")
]
table = bigquery.Table(table_ref=table, schema=schema)
table = bigquery_client.create_table(table=table) 
print(table.table_id)

#### Import csv from google storage

In [ ]:
uri = "gs://bigquery-lab/Housing.csv"

try:
    dataset = bigquery_client.dataset(dataset_id="bigquery_lab")
    # Create table
    table = dataset.table(table_id="housing")
    
    # Create schema
    job_config = bigquery.job.LoadJobConfig(
        autodetect=True,
        source_format = bigquery.SourceFormat.CSV,
        field_delimiter = ",",
        skip_leading_rows = 1
    )

    load_job = bigquery_client.load_table_from_uri(source_uris=uri, destination=table, job_config=job_config)

    load_job.result()
except Exception as err:
    print(err)

#### Import csv from local storage

In [ ]:
try:
    dataset = bigquery_client.dataset(dataset_id="bigquery_lab")
    table = dataset.table(table_id="titanic")
    job_config = bigquery.job.LoadJobConfig(
        autodetect=True,
        field_delimiter=";",
        source_format=bigquery.SourceFormat.CSV
    )
    
    with open("./titanic.csv", "rb") as source_file:
        load_job = bigquery_client.load_table_from_file(file_obj=source_file, 
                                                        destination=table, 
                                                        job_config=job_config)

    load_job.result()
except Exception as err:
    print(err)

#### Quering data with bigquery client

In [ ]:
query = """
    SELECT *
    FROM `dataengineer-310515.bigquery_lab.titanic`
    LIMIT 10
"""

job_config = bigquery.job.QueryJobConfig(use_query_cache=False)
results = bigquery_client.query(query=query, job_config=job_config).to_dataframe()

In [ ]:
results.head()

#### Download data to cloud storage

In [ ]:
destination_uri = "gs://bigquery-lab/titanic_backup.csv"
dataset = bigquery_client.dataset(dataset_id="bigquery_lab")
table = dataset.table("titanic")

extract_job = bigquery_client.extract_table(source=table, destination_uris=destination_uri)
extract_job.result()

#### External table

In [ ]:
# Get schema from table

table_name = "housing"
dataset_name = "bigquery_lab"
uri = "gs://bigquery-lab/Housing.csv"
project_id = "dataengineer-310515"

# Configure the external data source and query job.
table_ref = bigquery.DatasetReference(project=project_id, dataset_id=dataset_name).table(table_name)

schema = [
    bigquery.SchemaField(name="price", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="area", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="bedrooms", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="bathrooms", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="stories", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="mainroad", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="guestroom", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="basement", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="hotwaterheating", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="airconditioning", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="parking", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="prefarea", field_type="STRING", mode="NULLABLE"),
    bigquery.SchemaField(name="furnishingstatus", field_type="STRING", mode="NULLABLE")
]

table = bigquery.Table(table_ref, schema=schema)

external_config = bigquery.ExternalConfig("CSV")
external_config.source_uris = [uri]
external_config.options.field_delimiter = ","
external_config.options.skip_leading_rows = 1

table.external_data_configuration = external_config
# Create a permanent table linked to the GCS file
bigquery_client.delete_table(table=table, not_found_ok=True)
bigquery_client.create_table(table=table, exists_ok=True)

#### Get table schema

In [ ]:
table_name = "titanic"
dataset_name = "bigquery_lab"


table_ref = bigquery_client.get_dataset(dataset_ref=dataset_name).table(table_id=table_name)
table = bigquery_client.get_table(table=table_ref)
schema = table.schema

schema